In [ ]:
%load_ext autoreload
%autoreload 2
import nest_asyncio
nest_asyncio.apply()
    
from lmdeploy import TurbomindEngineConfig, pipeline
from lmdeploy.vl import load_image
model_name_or_path = "/home/yerong2/models/internlm-xcomposer2-vl-7b"
# backend_config = TurbomindEngineConfig(cache_max_entry_count=0.2)
pipe = pipeline(model_name_or_path,device_map="auto")

image = load_image('tiger.jpeg')
response = pipe(('describe this image', image))
print(response.text)
print(pipe.model)

In [1]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from repe.modeling_internlm_xcomposer2 import InternLMXComposer2ForCausalLM
# ckpt_path = "/home/yerong2/models/internlm-xcomposer2-vl-7b"
# tokenizer = AutoTokenizer.from_pretrained(ckpt_path, trust_remote_code=True)
# # Set `torch_dtype=torch.float16` to load model in float16, otherwise it will be loaded as float32 and might cause OOM Error.
# with torch.cuda.amp.autocast():
#     model = InternLMXComposer2ForCausalLM.from_pretrained(ckpt_path, torch_dtype=torch.float16, trust_remote_code=True, device_map='auto')
#     query = '<ImageHere>Please describe this image in detail.'
#     image = './tiger.jpeg'
#     response, _ = model.chat(tokenizer, query=query, image=image, history=[], do_sample=False)
# print(response)
import torch
from transformers import AutoModel, AutoTokenizer
from repe.modeling_internlm_xcomposer2 import InternLMXComposer2ForCausalLM

from repe.mllm_utils import auto_configure_device_map


model_name_or_path = "/home/yerong2/models/internlm-xcomposer2-vl-7b"

model = InternLMXComposer2ForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True).eval()

from accelerate import dispatch_model
device_map = auto_configure_device_map(3)
model = dispatch_model(model, device_map=device_map)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
model.tokenizer = tokenizer

query = '<ImageHere>Please describe this image'
# image = ['../tiger.jpeg']
image = 'tiger.jpeg'
with torch.autocast(device_type='cuda', dtype=torch.float16):
    # response, _ = model.chat(tokenizer, query, image, do_sample=False, num_beams=3, use_meta=True)
      response, _ = model.chat(tokenizer, query=query, image=image, history=[], do_sample=False)

print(response)

/home/yerong2/local/miniconda3/envs/mllm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yerong2/local/miniconda3/envs/mllm/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
You are using a model of type internlmxcomposer2 to instantiate a model of type internlm. This is not supported for all configurations of models and can yield errors.


Set max length to 4096
Position interpolate from 24x24 to 35x35


Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:08<00:00,  4.03s/it]
Some weights of InternLMXComposer2ForCausalLM were not initialized from the model checkpoint at /home/yerong2/models/internlm-xcomposer2-vl-7b and are newly initialized: ['vit.vision_tower.vision_model.post_layernorm.bias', 'vit.vision_tower.vision_model.post_layernorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{}
In the image, a majestic tiger is resting on a grassy field. The tiger's fur is a striking mix of orange and black stripes, with its head turned slightly to the left as it gazes directly into the camera. The background is a lush green field dotted with trees, providing a natural setting for this powerful creature. The tiger appears relaxed and at ease in its environment, embodying the serene atmosphere of the scene.
